<a href="https://colab.research.google.com/github/usmn777/RESNET/blob/main/ResNetArc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

In [2]:
class block(nn.Module):
  def __init__(self,in_channels,intermediate_channels,identity_downsample=None,stride=1):
    super(block,self).__init__()
    self.expansion = 4
    self.conv1 = nn.Conv2d(in_channels,intermediate_channels,kernel_size =1, stride = 1,padding = 0, bias = False)
    self.bn1 = nn.BatchNorm2d(intermediate_channels)
    self.conv2 = nn.Conv2d(intermediate_channels, intermediate_channels, kernel_size=3, stride = stride, padding = 1, bias = False)
    self.bn2 = nn.BatchNorm2d(intermediate_channels)
    self.conv3 = nn.Conv2d(intermediate_channels,intermediate_channels * self.expansion,kernel_size = 1, stride=1,padding=0, bias=False)
    self.bn3=nn.BatchNorm2d(intermediate_channels * self.expansion)
    self.relu=nn.ReLU()
    self.identity_downsample = identity_downsample
    self.stride = stride

In [3]:
def forward(self,x):
  identity = x.clone()
  x = self.conv1(x)
  x = self.bn1(x)
  x = self.relu(x)
  x= self.conv2(x)
  x = self.bn2(x)
  x = self.relu(x)
  x = self.conv3(x)
  x = self.bn3(x)
  if self.identity_downsample is not None:
    identity = self.identity_downsample(identity)
  x += identity
  x = self.relu(x)
  return x

In [4]:
class ResNet(nn.Module):
  def __init__(self,block,layers,image_channels,num_classes):
    super(ResNet,self).__init__()
    self.in_channels = 64
    self.conv1 = nn.Conv2d(image_channels,64,kernel_size=7, stride=2,bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu= nn.Relu()
    self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    self.layer1 = self._make_layer(block,layers[0],intermediate_channels=64,stride=1)
    self.layer2 = self._make_layer(block,layers[1],intermediate_channels=128, stride=2)
    self.layer3 = self._make_layer(block,layers[2],intermediate_channels=256,stride=2)
    self.layer4 = self._make_layer(block,layers[3],intermediate_channels=512,stride=2)
    self.avgpool = nn.AdaptiveAvgPool2d(1,1)
    self.fc = nn.Linear(512 * 4 * num_classes)